## Imports

In [71]:
import pandas as pd
import numpy as np
from IPython import display
from pymongo import MongoClient
import os
import re
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize
from sklearn.decomposition import NMF
from nltk.util import ngrams
import operator
from gensim import corpora, models, similarities, matutils
from sklearn import datasets
import json

# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


display.clear_output(wait=True)
from config import user_name,password,ip
from epub_conversion.utils import open_book

## Functions

In [115]:
def clean_text(row):
    text = row['content'].lower()
    text = text.strip('\n')
    return text

In [12]:
def book_word_count(book,n,stoplist=stoplist):
    text = clean_text(book)
    words = [''.join(words) for words in text.split()]
    title = book['title']
    counter = Counter()
    n = n
    words = [w for w in words if w not in stoplist]
    bigrams = ngrams(words, n)
    counter += Counter(bigrams)
    sorted_counter = sorted(counter.items(), key=operator.itemgetter(1),reverse=True)
    return title, sorted_counter

In [68]:
def book_cv(dtbooks,stoplist):
    cv = CountVectorizer(stop_words=stoplist,token_pattern="\\b[a-z][a-z]+\\b")
    print(type(dtbooks[0]))
    cv.fit(dtbooks)
    x = cv.transform(dtbooks)
    x_back = x.toarray()
    df = pd.DataFrame(x_back, columns=cv.get_feature_names())
    counts = cv.transform(dtbooks).transpose()
    print(counts.shape)
    corpus = matutils.Sparse2Corpus(counts)
    id2word = dict((v, k) for k, v in cv.vocabulary_.items())
    return df,corpus,id2word

## Stoplist

In [55]:
stoplist = stopwords.words('english')
stoplist += ['.', ',', '(', ')', "'", '"']
stoplist = set(stoplist)

In [3]:
configString = "mongodb://"+user_name+":"+password+"@"+ip+"/fletcher"

In [4]:
# client = MongoClient(configString) # defaults to port 27017

# db = client.fletcher

# # print the number of documents in a collection
# #print (db.collection_names())

In [6]:
#print(db.create_collection('books'))

In [5]:
db.collection_names()

['books', 'test']

In [6]:
!pwd

/Users/xavier/dev/metis/fletcher


**Create a dictionary from all books**

In [7]:
# book_list = []
# book_dict = {}
# path = "/Users/xavier/dev/metis/fletcher/books/"
# for file in os.listdir(path):
#     if file.endswith(".txt"):
#         clean_name = file.replace(" - Stephen King.txt","")
#         book_dict[clean_name] = open(path+file, "r").read()
#         book_list.append(clean_name)

In [116]:
# # Insert the books into mongo db
# clean_list = []
# for k,v in book_dict.items():
#     try:
#         year = re.search("[Cc]opyright ©\s*.*_*(\d{4}).*Stephen King|[Cc]opyright ©\s.*Stephen King.*_*(\d{4})|[Cc]opyright ©\s*.*_*(\d{4}).*Richard Bachman|[Cc]opyright ©\s.*Richard Bachman.*_*(\d{4})",v).group(0) # get copyright year from book text
#         year = re.search("(\d{4})",year).group(0)
#     except:
#         year = ""
#     try:
#         isbn = re.search(".*ISBN+:*(\d*.*)",v)[1].split(" ")
#         isbn = max(isbn, key=len)
#     except:
#         isbn = ""
#     try:
#         start = v.find('******start_of_file******')+25
#         end = v.find('******end_of_file******')
#         text = v[start:end]
#     except:
#         text = ""
        
#     doc = {"title":k,"year":year,'isbn':isbn,"content":text}
#     clean_list.append(doc)
#     #print(doc['title'],doc['isbn'])
#     #print(doc['title'],doc['year'])
#     #db.books.insert_one(doc)

In [110]:
len(clean_list)

68

In [111]:
#pd.DataFrame(a, index=['i',])
df = pd.DataFrame(clean_list)

In [112]:
df.to_pickle('books.pkl')

In [113]:
df.columns.tolist()

['content', 'isbn', 'title', 'year']

In [114]:
df.head()

,content,isbn,title,year
0,d by “Duel”\n\nJoe Hill and Stephen King\n\n\n...,9780062215956,Throttle,2009
1,TS\n\n\n\nCover Page\n\nTitle Page\n\n\n\nIntr...,978-0-385-52884-9,Night Shift,1976
2,this Scribner eBook.\n\n\n\n* * *\n\n\n\nSign...,0-7432-0467-0,Riding the Bullet,2000
3,Page\n\nCopyright Page\n\nDedication\n\n\n\n\...,978-1-101-13813-7,Roadwork,1981
4,dication\n\nIntroduction\n\nAuthor’s Note\n\n\...,978-0-385-52822-1,Salem's Lot,1975


In [123]:
df['content'] = df.content.apply(lambda x: x.lower())
df['content'] = df.content.apply(lambda x: x.strip("\n"))

**Now that the books are in mongo lets try to play with the text data**

In [11]:
# db.books.remove({})

In [19]:
words = [''.join(words) for words in gs_text.split()]
# vectorizer = TfidfVectorizer(stop_words=stop, ngram_range=(1))
# doc_vectors = vectorizer.fit_transform(documents)

In [72]:
cv = CountVectorizer(stop_words=stoplist)
cv.fit(gs_sentences)
x = cv.transform(gs_sentences)

In [22]:
x_back = x.toarray()

In [23]:
gunslingerDF = pd.DataFrame(x_back, columns=cv.get_feature_names())

In [24]:
gunslingerDF.head()

,000,10,100,10014,101,14,14645,19,1966,1967,...,zachary,zealot,zealous,zealously,zenith,zigzag,zigzagging,zoltan,zone,zorro
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
nmf = NMF(n_components=10, init='random')
test = nmf.fit_transform(gunslingerDF)

In [28]:
type(test)

numpy.ndarray

In [29]:
test[:5]

array([[  5.57641644e-03,   0.00000000e+00,   1.41894660e-02,
          1.41393186e-03,   1.99525780e-03,   6.45673952e-03,
          9.92247321e-03,   1.50025622e-03,   8.59849872e-03,
          3.13692542e-04],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.40333238e-01,   0.00000000e+00,
          1.01013836e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  8.09998758e-03,   3.71753529e-03,   1.31278385e-04,
          3.63206140e-03,   1.53856971e-03,   4.92148606e-03,
          3.87489934e-02,   5.82179131e-05,   1.09081363e-03,
          3.06805194e-03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   7.09315934e-02,
          1.44315156e-02,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  2.02208521e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.77445461e+00

In [30]:
gunslingerDF.columns

Index(['000', '10', '100', '10014', '101', '14', '14645', '19', '1966', '1967',
       ...
       'zachary', 'zealot', 'zealous', 'zealously', 'zenith', 'zigzag',
       'zigzagging', 'zoltan', 'zone', 'zorro'],
      dtype='object', length=7505)

In [32]:
gBlob.words

WordList(['n', 'names', 'characters', 'places', 'and', 'incidents', 'are', 'either', 'the', 'product', 'of', 'the', 'author', '’', 's', 'imagination', 'or', 'are', 'used', 'fictitiously', 'and', 'any', 'resemblance', 'to', 'actual', 'persons', 'living', 'or', 'dead', 'business', 'establishments', 'events', 'or', 'locales', 'is', 'entirely', 'coincidental', 'the', 'gunslinger', 'a', 'viking', 'penguin', 'book', 'published', 'by', 'arrangement', 'with', 'the', 'author', 'all', 'rights', 'reserved', 'copyright', '©', '1982', '2003', 'by', 'stephen', 'king', 'this', 'book', 'may', 'not', 'be', 'reproduced', 'in', 'whole', 'or', 'part', 'by', 'mimeograph', 'or', 'any', 'other', 'means', 'without', 'permission', 'making', 'or', 'distributing', 'electronic', 'copies', 'of', 'this', 'book', 'constitutes', 'copyright', 'infringement', 'and', 'could', 'subject', 'the', 'infringer', 'to', 'criminal', 'and', 'civil', 'liability', 'for', 'information', 'address', 'the', 'berkley', 'publishing', 'gr

Test Splitting the text for just the story content

In [95]:
# start = gs_text.find('******start_of_file******')+25
# end = gs_text.find('******end_of_file******')
# gs_text = gs_text[start:end]

In [105]:
# gs_text[-20:-1]

'ble final battle.\n\n'

In [8]:
words = [''.join(words) for words in gs_text.split()]

NameError: name 'gs_text' is not defined

In [52]:
counter = Counter()
n = 1
words = [w for w in words if w not in stoplist]
bigrams = ngrams(words, n)
counter += Counter(bigrams)

sorted_counter = sorted(counter.items(), key=operator.itemgetter(1),reverse=True)
# for word, count in gBlob.word_counts.items():
#     print("%15s %i" % (word, count))

In [53]:
sorted_counter

[(('gunslinger',), 531),
 (('man',), 243),
 (('boy',), 236),
 (('one',), 218),
 (('like',), 201),
 (('would',), 186),
 (('“i',), 169),
 (('black',), 156),
 (('said.',), 151),
 (('looked',), 147),
 (('back',), 143),
 (('could',), 136),
 (('said',), 129),
 (('jake',), 120),
 (('made',), 103),
 (('don’t',), 99),
 (('even',), 92),
 (('him.',), 92),
 (('it.',), 90),
 (('him,',), 89),
 (('eyes',), 88),
 (('seemed',), 87),
 (('came',), 85),
 (('roland',), 85),
 (('face',), 83),
 (('went',), 82),
 (('felt',), 81),
 (('might',), 80),
 (('way',), 77),
 (('still',), 77),
 (('know',), 76),
 (('time',), 75),
 (('never',), 74),
 (('come',), 74),
 (('“you',), 73),
 (('see',), 71),
 (('first',), 69),
 (('two',), 69),
 (('thought',), 68),
 (('cort',), 68),
 (('began',), 66),
 (('hands',), 66),
 (('said,',), 66),
 (('long',), 65),
 (('it,',), 65),
 (('last',), 65),
 (('go',), 64),
 (('almost',), 63),
 (('head',), 60),
 (('didn’t',), 59),
 (('saw',), 59),
 (('perhaps',), 59),
 (('old',), 58),
 (('three',

In [70]:
all_books = db.books.find({})

In [59]:
df = pd.DataFrame(columns=['title','isbn','text'])
for book in all_books:
    df2 = 

Throttle
Night Shift
Riding the Bullet
Roadwork
Salem's Lot
Misery
Everything's Eventual_ 14 Dark Tales
Song of Susannah
Wizard and Glass
Thinner
Hearts in Atlantis
Dolores Claiborne
Finders Keepers
Insomnia
Different Seasons
Dead Zone, The
Bag of Bones_ A Novel
Tommyknockers, The
Wind Through the Keyhole_ A Dark Tower Novel, The
Mist, The
Gerald's Game
Nightmares & Dreamscapes
Big Driver
Good Marriage, A
From a Buick 8_ A Novel
Revival
Eyes of the Dragon, The
Waste Lands, The
Gunslinger, The
Girl Who Loved Tom Gordon_ A Novel, The
Talisman_ A Novel, The
Mr. Mercedes
Rose Madder
Stand, The
Four Past Midnight
Dark Half, The
Regulators, The
Cell
Face in the Crowd, A
Long Walk, The
Carrie
Skeleton Crew
Blockade Billy
11_22_63
Blaze_ A Novel
Joyland
Doctor Sleep_ A Novel
Drawing of the Three, The
Full Dark, No Stars
Mile 81
Shining, The
Desperation
Dreamcatcher_ A Novel
Needful Things_ The Last Castle Rock Story
Green Mile_ The Complete Serial Novel, The
Running Man, The
Just After Sunset_

In [14]:
book_counts = {}
for book in all_books:
    title,counts = book_word_count(book,1)
    book_counts[title] = counts

In [16]:
clean_books = []
for book in all_books:
    text = clean_text(book)
    clean_books.append(text)

In [17]:
book_list = sorted(book_list)

In [29]:
dt_books = ['Gunslinger, The','Drawing of the Three, The','Waste Lands, The',
            'Wizard and Glass','Wolves of the Calla','Song of Susannah','Dark Tower, The']

In [30]:
print(dt_books)

['Gunslinger, The', 'Drawing of the Three, The', 'Waste Lands, The', 'Wizard and Glass', 'Wolves of the Calla', 'Song of Susannah', 'Dark Tower, The']


In [40]:
all_books = db.books.find({})

In [42]:
all_text = []
for book in all_books:
    text = clean_text(book)
    all_text.append(text)

In [25]:
dt_text = []
for book in all_books:
    if book['title'] in dt_books:
        text  = clean_text(book)
        name = book['title']
        print(book['title'])
        dt_text.append(text)
    #else:
        #print("Haha Fuck you!")

Song of Susannah
Wizard and Glass
Waste Lands, The
Gunslinger, The
Drawing of the Three, The
Dark Tower, The
Wolves of the Calla


In [64]:
len(dt_text)

7

In [69]:
df,corp,id2word = book_cv(dt_text,stoplist)

<class 'str'>
(29598, 7)


In [ ]:
df.shape

### LDA

In [45]:
# lda = models.LdaModel(corpus=corp, num_topics=10, id2word=id2word, passes=10)

2017-11-02 17:40:37,750 : INFO : using symmetric alpha at 0.1
2017-11-02 17:40:37,751 : INFO : using symmetric eta at 2.13193251154e-06
2017-11-02 17:40:37,826 : INFO : using serial LDA version on this node
2017-11-02 17:40:58,603 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 7 documents, updating model once every 7 documents, evaluating perplexity every 7 documents, iterating 50x with a convergence threshold of 0.001000
2017-11-02 17:41:47,044 : INFO : -17.591 per-word bound, 197481.1 perplexity estimate based on a held-out corpus of 7 documents with 1304555 words
2017-11-02 17:41:47,046 : INFO : PROGRESS: pass 0, at document #7/7
2017-11-02 17:41:48,766 : INFO : topic #7 (0.100): 0.005*"said" + 0.005*"roland" + 0.004*"eddie" + 0.003*"like" + 0.003*"jake" + 0.003*"one" + 0.003*"would" + 0.002*"back" + 0.002*"susannah" + 0.002*"could"
2017-11-02 17:41:48,772 : INFO : topic #1 (0.100): 0.006*"said" + 0.004*"roland" + 0.004*"one" + 0.

2017-11-02 17:44:00,647 : INFO : topic diff=0.267182, rho=0.377964
2017-11-02 17:44:23,423 : INFO : -11.579 per-word bound, 3060.3 perplexity estimate based on a held-out corpus of 7 documents with 1304555 words
2017-11-02 17:44:23,424 : INFO : PROGRESS: pass 6, at document #7/7
2017-11-02 17:44:30,039 : INFO : topic #7 (0.100): 0.001*"said" + 0.001*"roland" + 0.001*"would" + 0.001*"eddie" + 0.001*"one" + 0.001*"like" + 0.001*"jake" + 0.001*"susannah" + 0.000*"back" + 0.000*"could"
2017-11-02 17:44:30,044 : INFO : topic #3 (0.100): 0.000*"said" + 0.000*"eddie" + 0.000*"one" + 0.000*"like" + 0.000*"roland" + 0.000*"would" + 0.000*"jake" + 0.000*"back" + 0.000*"susannah" + 0.000*"could"
2017-11-02 17:44:30,048 : INFO : topic #4 (0.100): 0.000*"roland" + 0.000*"said" + 0.000*"eddie" + 0.000*"one" + 0.000*"would" + 0.000*"could" + 0.000*"like" + 0.000*"know" + 0.000*"jake" + 0.000*"gunslinger"
2017-11-02 17:44:30,052 : INFO : topic #8 (0.100): 0.006*"said" + 0.006*"roland" + 0.004*"one" + 

In [46]:
# lda.print_topics()

2017-11-02 17:45:59,374 : INFO : topic #0 (0.100): 0.005*"gunslinger" + 0.003*"said" + 0.003*"boy" + 0.003*"man" + 0.002*"one" + 0.002*"like" + 0.002*"would" + 0.002*"back" + 0.002*"black" + 0.001*"jake"
2017-11-02 17:45:59,380 : INFO : topic #1 (0.100): 0.000*"said" + 0.000*"roland" + 0.000*"one" + 0.000*"would" + 0.000*"like" + 0.000*"eddie" + 0.000*"back" + 0.000*"could" + 0.000*"looked" + 0.000*"jake"
2017-11-02 17:45:59,384 : INFO : topic #2 (0.100): 0.005*"said" + 0.004*"eddie" + 0.004*"roland" + 0.003*"one" + 0.003*"susannah" + 0.003*"mia" + 0.002*"like" + 0.002*"would" + 0.002*"jake" + 0.002*"back"
2017-11-02 17:45:59,388 : INFO : topic #3 (0.100): 0.000*"said" + 0.000*"eddie" + 0.000*"one" + 0.000*"like" + 0.000*"roland" + 0.000*"would" + 0.000*"jake" + 0.000*"back" + 0.000*"susannah" + 0.000*"could"
2017-11-02 17:45:59,392 : INFO : topic #4 (0.100): 0.000*"roland" + 0.000*"said" + 0.000*"eddie" + 0.000*"one" + 0.000*"would" + 0.000*"could" + 0.000*"like" + 0.000*"know" + 0.00

[(0,
  '0.005*"gunslinger" + 0.003*"said" + 0.003*"boy" + 0.003*"man" + 0.002*"one" + 0.002*"like" + 0.002*"would" + 0.002*"back" + 0.002*"black" + 0.001*"jake"'),
 (1,
  '0.000*"said" + 0.000*"roland" + 0.000*"one" + 0.000*"would" + 0.000*"like" + 0.000*"eddie" + 0.000*"back" + 0.000*"could" + 0.000*"looked" + 0.000*"jake"'),
 (2,
  '0.005*"said" + 0.004*"eddie" + 0.004*"roland" + 0.003*"one" + 0.003*"susannah" + 0.003*"mia" + 0.002*"like" + 0.002*"would" + 0.002*"jake" + 0.002*"back"'),
 (3,
  '0.000*"said" + 0.000*"eddie" + 0.000*"one" + 0.000*"like" + 0.000*"roland" + 0.000*"would" + 0.000*"jake" + 0.000*"back" + 0.000*"susannah" + 0.000*"could"'),
 (4,
  '0.000*"roland" + 0.000*"said" + 0.000*"eddie" + 0.000*"one" + 0.000*"would" + 0.000*"could" + 0.000*"like" + 0.000*"know" + 0.000*"jake" + 0.000*"gunslinger"'),
 (5,
  '0.006*"roland" + 0.005*"said" + 0.005*"eddie" + 0.005*"one" + 0.004*"jake" + 0.003*"like" + 0.003*"would" + 0.003*"could" + 0.003*"back" + 0.002*"susannah"'),
 (6

In [47]:
# # Transform the docs from the word space to the topic space (like "transform" in sklearn)
# lda_corpus = lda[corp]
# lda_docs = [doc for doc in lda_corpus]

In [48]:
# lda_docs[:15]

[[(2, 0.99999309930389702)],
 [(8, 0.99999659776824634)],
 [(5, 0.99999504978884213)],
 [(0, 0.9999867660232642)],
 [(5, 0.80863682693916183), (9, 0.19135691336193036)],
 [(5, 0.9999967903172694)],
 [(8, 0.99999642158611446)]]

In [49]:
# lda.log_perplexity

<bound method LdaModel.log_perplexity of <gensim.models.ldamodel.LdaModel object at 0x1a29cd38d0>>

In [50]:
# corp2,id2word2 = book_cv(all_text,stoplist)

<class 'str'>
(2680972, 67)


In [52]:
# lda2 = models.LdaModel(corpus=corp2, num_topics=10, id2word=id2word2, passes=10)

2017-11-02 18:59:21,423 : INFO : using symmetric alpha at 0.1
2017-11-02 18:59:21,425 : INFO : using symmetric eta at 3.72999046614e-07
2017-11-02 18:59:21,803 : INFO : using serial LDA version on this node
2017-11-02 19:01:16,931 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 67 documents, updating model once every 67 documents, evaluating perplexity every 67 documents, iterating 50x with a convergence threshold of 0.001000
2017-11-02 19:06:27,225 : INFO : -18.245 per-word bound, 310611.4 perplexity estimate based on a held-out corpus of 67 documents with 10236337 words
2017-11-02 19:06:27,227 : INFO : PROGRESS: pass 0, at document #67/67
2017-11-02 19:06:41,693 : INFO : topic #0 (0.100): 0.005*"said" + 0.003*"would" + 0.003*"like" + 0.003*"one" + 0.002*"time" + 0.002*"could" + 0.002*"back" + 0.002*"looked" + 0.002*"go" + 0.002*"way"
2017-11-02 19:06:41,733 : INFO : topic #5 (0.100): 0.005*"said" + 0.004*"one" + 0.003*"like" + 0.003

2017-11-02 19:23:20,101 : INFO : topic diff=0.286577, rho=0.377964
2017-11-02 19:26:11,274 : INFO : -12.282 per-word bound, 4981.2 perplexity estimate based on a held-out corpus of 67 documents with 10236337 words
2017-11-02 19:26:11,276 : INFO : PROGRESS: pass 6, at document #67/67
2017-11-02 19:26:53,785 : INFO : topic #9 (0.100): 0.000*"said" + 0.000*"one" + 0.000*"like" + 0.000*"back" + 0.000*"could" + 0.000*"would" + 0.000*"know" + 0.000*"thought" + 0.000*"right" + 0.000*"looked"
2017-11-02 19:26:53,833 : INFO : topic #2 (0.100): 0.003*"said" + 0.002*"one" + 0.002*"dan" + 0.002*"like" + 0.002*"back" + 0.001*"would" + 0.001*"could" + 0.001*"abra" + 0.001*"know" + 0.001*"little"
2017-11-02 19:26:53,871 : INFO : topic #7 (0.100): 0.000*"said" + 0.000*"one" + 0.000*"back" + 0.000*"would" + 0.000*"thought" + 0.000*"could" + 0.000*"like" + 0.000*"know" + 0.000*"right" + 0.000*"see"
2017-11-02 19:26:53,916 : INFO : topic #3 (0.100): 0.001*"said" + 0.001*"wesley" + 0.001*"one" + 0.000*"li

In [53]:
# lda2.print_topics()

2017-11-02 19:37:40,060 : INFO : topic #0 (0.100): 0.005*"roland" + 0.004*"said" + 0.004*"one" + 0.003*"would" + 0.002*"like" + 0.002*"back" + 0.002*"eddie" + 0.002*"could" + 0.002*"susannah" + 0.002*"jake"
2017-11-02 19:37:40,086 : INFO : topic #1 (0.100): 0.005*"said" + 0.004*"one" + 0.003*"like" + 0.003*"back" + 0.003*"would" + 0.002*"could" + 0.002*"know" + 0.002*"little" + 0.002*"looked" + 0.002*"time"
2017-11-02 19:37:40,132 : INFO : topic #2 (0.100): 0.002*"said" + 0.002*"dan" + 0.002*"one" + 0.002*"like" + 0.001*"back" + 0.001*"abra" + 0.001*"would" + 0.001*"could" + 0.001*"know" + 0.001*"little"
2017-11-02 19:37:40,180 : INFO : topic #3 (0.100): 0.001*"wesley" + 0.000*"kindle" + 0.000*"robbie" + 0.000*"said" + 0.000*"ur" + 0.000*"one" + 0.000*"like" + 0.000*"ellen" + 0.000*"back" + 0.000*"know"
2017-11-02 19:37:40,220 : INFO : topic #4 (0.100): 0.003*"said" + 0.002*"one" + 0.002*"danny" + 0.002*"like" + 0.002*"back" + 0.002*"would" + 0.002*"jack" + 0.001*"could" + 0.001*"time"

[(0,
  '0.005*"roland" + 0.004*"said" + 0.004*"one" + 0.003*"would" + 0.002*"like" + 0.002*"back" + 0.002*"eddie" + 0.002*"could" + 0.002*"susannah" + 0.002*"jake"'),
 (1,
  '0.005*"said" + 0.004*"one" + 0.003*"like" + 0.003*"back" + 0.003*"would" + 0.002*"could" + 0.002*"know" + 0.002*"little" + 0.002*"looked" + 0.002*"time"'),
 (2,
  '0.002*"said" + 0.002*"dan" + 0.002*"one" + 0.002*"like" + 0.001*"back" + 0.001*"abra" + 0.001*"would" + 0.001*"could" + 0.001*"know" + 0.001*"little"'),
 (3,
  '0.001*"wesley" + 0.000*"kindle" + 0.000*"robbie" + 0.000*"said" + 0.000*"ur" + 0.000*"one" + 0.000*"like" + 0.000*"ellen" + 0.000*"back" + 0.000*"know"'),
 (4,
  '0.003*"said" + 0.002*"one" + 0.002*"danny" + 0.002*"like" + 0.002*"back" + 0.002*"would" + 0.002*"jack" + 0.001*"could" + 0.001*"time" + 0.001*"know"'),
 (5,
  '0.004*"said" + 0.004*"one" + 0.003*"like" + 0.003*"back" + 0.002*"would" + 0.002*"could" + 0.002*"time" + 0.002*"know" + 0.002*"little" + 0.002*"right"'),
 (6,
  '0.005*"said" 

In [54]:
# print(stoplist)

{'shan', 'too', 'wasn', 'under', 'about', 'than', 'can', 'further', 'above', 'o', 't', 'herself', 'again', 'at', 'was', 'didn', 'we', 'isn', 'does', 'me', 'i', 'couldn', 'the', 'been', 'she', 'her', "'", 'as', 'those', 'then', 'until', 'these', 'when', 'both', 'y', 'before', 'they', 'same', 're', ')', 'each', 'were', 'are', 'mightn', 'mustn', 'nor', 'very', 'of', 'how', 'don', 'any', 'ours', 'your', 'there', 'our', 'm', 'being', 'aren', 'do', 'so', 'am', 'some', 'such', '.', 'did', 'up', 'through', 'not', 'myself', 'you', 'should', 'between', 'out', 'where', 'to', 'weren', 'for', 's', 'haven', 'yourself', 'whom', 'down', '(', 'from', 'my', 'yours', 'ain', 'is', 'yourselves', 'their', 'which', 'what', 've', 'had', 'needn', 'himself', 'more', 'an', 'few', 'here', 'a', 'but', 'below', 'if', '"', 'why', 'doing', 'all', 'on', 'itself', 'just', 'hadn', 'them', 'own', 'hasn', 'only', 'will', 'wouldn', 'be', 'him', ',', 'shouldn', 'by', 'against', 'with', 'it', 'into', 'he', 'theirs', 'ourselv